# 6. We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression co- efficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the glm() function. Do not forget to set a random seed before beginning your analysis.

In [13]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.utils import resample

In [2]:
df = pd.read_csv('/Volumes/work/sem_1/MTH522/data/Default.csv')
df.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [4]:
df['default'] = np.where(df['default'] == 'Yes', 1, 0)
df

,default,student,balance,income
0,0,No,729.526495,44361.625074
1,0,Yes,817.180407,12106.134700
2,0,No,1073.549164,31767.138947
3,0,No,529.250605,35704.493935
4,0,No,785.655883,38463.495879
...,...,...,...,...
9995,0,No,711.555020,52992.378914
9996,0,No,757.962918,19660.721768
9997,0,No,845.411989,58636.156984
9998,0,No,1569.009053,36669.112365


## (a) Using the summary() and glm() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.

In [7]:
np.random.seed(17)
x = df.iloc[:, 2:]
X = sm.add_constant(x)
Y = df.iloc[:, :1]

model = sm.Logit(Y, X).fit()
model.summary()


Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Sun, 30 Oct 2022   Pseudo R-squ.:                  0.4594
Time:                        07:52:46   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0.000     24.835      0.000       0.005       0.006
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [8]:
model.bse['income']


4.985245461753038e-06

In [9]:
model.bse['balance']


0.00022738138476844578

In [10]:
print('Coefficient of income and balance is', model.params.values[[1, 2]])


Coefficient of income and balance is [5.64710295e-03 2.08089755e-05]


**Observation:**
1. The standard error from the model that is generated is 2.27x10^-4 for "balance" and 4.985×10^−6 for "income".


## (b) Write a function,boot.fn(),that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

In [11]:
def boot_fn(data, obs):
    X = sm.add_constant(data.iloc[:obs, 2:])
    Y = data.iloc[:obs, :1]

    model = sm.Logit(Y, X).fit()

    return model.bse['income'], model.bse['balance']

## (c) Use the boot() function together with your boot.fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.


In [14]:
np.random.seed(312)
income = []
balance = []
for i in range(1000):
    sample = resample(df)
    inc, bal = boot_fn(sample, 7000)
    income.append(inc)
    balance.append(bal)

Optimization terminated successfully.
         Current function value: 0.076691
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.094605
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.083446
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.085226
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076326
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077480
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.080803
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078484
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.083256
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

In [15]:
df_error = pd.DataFrame(income, columns=['Income'])
df_error['Balance'] = balance
df_error

,Income,Balance
0,0.000006,0.000294
1,0.000005,0.000238
2,0.000006,0.000252
3,0.000006,0.000251
4,0.000006,0.000270
...,...,...
995,0.000006,0.000276
996,0.000006,0.000267
997,0.000006,0.000255
998,0.000006,0.000261


In [17]:
print('Standard error of mean for income: ', np.mean(df_error['Income']))
print('Standard error of mean for balance: ', np.mean(df_error['Balance']))

Standard error of mean for income:  5.986813530229936e-06
Standard error of mean for balance:  0.00027423674240413466


## (d) Comment on the estimated standard errors obtained using the glm() function and using your bootstrap function.


**Observation:**
1. Standard error mean after doing bootstrap is almost similar to result what we have got earlier.
